In [ ]:
import h5py
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# reconfigure datasets
student_id_last_digit = 4
new_train_size = 160 + student_id_last_digit

# split the original training set
new_train_X = train_X[:new_train_size]
new_train_Y = train_Y[:new_train_size]
new_test_X = train_X[new_train_size:]
new_test_Y = train_Y[new_train_size:]

# create DataLoaders for new dataset
new_train_dataset = TensorDataset(new_train_X, new_train_Y)
new_test_dataset = TensorDataset(new_test_X, new_test_Y)

new_train_loader = DataLoader(new_train_dataset, batch_size=32, shuffle=True)
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

# define new model, loss and optimizer
model = LogisticRegressionModel(input_dim=new_train_X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, Y_batch in new_train_loader:
        predictions = model(X_batch)
        loss = criterion(predictions, Y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Evaluate on new test set
model.eval()
with torch.no_grad():
    predictions = (model(new_test_X) > 0.5).float()
    new_test_accuracy = (predictions == new_test_Y).float().mean().item()
    print(f"Modified Test Accuracy: {new_test_accuracy:.4f}")

# Find indices of incorrect predictions
incorrect_indices = torch.where(predictions.flatten() != new_test_Y.flatten())[0].numpy()
print(f"Incorrect Predictions Indices: {incorrect_indices}")

# Display failed cases
plt.figure(figsize=(10, 10))
for i, idx in enumerate(incorrect_indices[:4]):
    plt.subplot(2, 2, i + 1)
    plt.imshow(new_test_X[idx].reshape(64, 64, 3).numpy())
    plt.title(f"Index: {idx}, Pred: {int(predictions[idx])}, True: {int(new_test_Y[idx])}")
    plt.axis('off')
plt.show()